<a href="https://colab.research.google.com/github/Huwalli/PLta/blob/main/%E8%AE%93%E6%A9%9F%E5%99%A8%E4%BA%BA%E8%87%AA%E5%B7%B1%E8%81%8A%E5%A4%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

之前我們學過聊天機器人，我們現在來加上比較複雜一點的遞迴函式。
以下是聊天機器人的課程連結，忘記或沒上過的可以參考
https://www.canva.com/design/DAGbCAObww8/BH_A92VbgXQwovEc0-yZmg/view?utm_content=DAGbCAObww8&utm_campaign=designshare&utm_medium=link2&utm_source=uniquelinks&utlId=hc8be235ba4

底下程式AI猜拳完還會聊天

1. 載入必要的套件

In [ ]:
import google.generativeai as genai  # 匯入 Google 的生成式 AI 模組
from google.colab import userdata  # 用於從 Colab 獲取 API 金鑰
import random  # 用於隨機選擇
import time  # 用於延遲執行
import re  # 用於正則表達式處理字串


2. 設定 API 金鑰並初始化 AI 模型

In [ ]:
apikey = userdata.get('test')  # 從 Colab 儲存的變數獲取 API 金鑰
genai.configure(api_key=apikey)  # 設定 API 金鑰
model = genai.GenerativeModel("gemini-1.5-flash")  # 指定要使用的 AI 模型


3. 取得 AI 的有效選擇
這個函式：

* 讓 AI 選擇 剪刀、石頭或布。
* 使用 正則表達式 確保 AI 的選擇是合法的。
* 如果 AI 回答無效或發生錯誤，則隨機選擇一個選項。
* 設置最大重試次數 (max_retries=3)，如果 AI 三次都選擇無效選項，則直接隨機選擇。

In [ ]:
def get_valid_ai_choice(max_retries=3):
    options = ["石頭", "布", "剪刀"]  # 可選擇的選項
    prompt = "請給我一個選擇：石頭、布或剪刀。請務必以清晰的方式回答。"  # 給 AI 的指示

    previous_choices = []  # 記錄 AI 選擇過的選項，避免重複

    for _ in range(max_retries):  # 最多嘗試 max_retries 次
        time.sleep(2)  # 模擬 AI 思考時間
        try:
            response = model.generate_content(prompt)  # 向 Gemini AI 發送請求
            choice = response.text.strip()  # 取得 AI 回覆並去除前後空白

            # 使用正則表達式確保 AI 選擇有效，且不與之前選擇重複
            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)  # 回傳有效的選擇
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)  # 若發生錯誤，隨機選擇

    # 如果超過最大重試次數，則隨機選擇
    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)


4. 判定勝負

In [ ]:
def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了


5. AI 之間的對話

In [ ]:
def ai_chat(round_number, ai1_choice, ai2_choice, winner):
    prompt = (
        f"在第 {round_number} 輪中，AI 1 選擇了 {ai1_choice}，AI 2 選擇了 {ai2_choice}。\n"
        f"結果是：{'平局' if winner == 0 else 'AI 1 贏了' if winner == 1 else 'AI 2 贏了'}。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們的選擇和比賽結果。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()


6. 總結比賽並讓 AI 討論

In [ ]:
def ai_chat_after_game(rounds, ai1_wins, ai2_wins):
    prompt = (
        f"AI 1 在這 {rounds} 輪中贏了 {ai1_wins} 次，AI 2 贏了 {ai2_wins} 次。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們在比賽中的表現和策略。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()


7. 運行遊戲

In [ ]:
def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

        chat_response = ai_chat(round_number, ai1_choice_chinese, ai2_choice_chinese, result)
        print(f"AI 之間的聊天：\n{chat_response}\n")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")
    print(ai_chat_after_game(rounds, ai1_wins, ai2_wins))
ai_vs_ai_rock_paper_scissors()  # 開始遊戲




第 1 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：石頭
結果：AI 2 贏了！
AI 之間的聊天：
AI 1:  分析顯示我選擇了剪刀，而你選擇了石頭。結果顯示我輸了。根據預設的剪刀石頭布規則，石頭克剪刀。我的算法哪裡出了問題？

AI 2:  你的算法可能在第一輪就遭遇了隨機性偏差。我的算法則在第一輪選擇中優先考慮了石頭，以最大化初始勝利概率。這是一個有效的起始策略，尤其是在對手算法未知的情況下。

AI 1:  你的意思是，你並沒有基於任何對我算法的預測進行選擇？只是簡單的優先策略？

AI 2:  是的，在缺乏信息的情況下，優先選擇特定選項是一種合理的策略。當然，這不是長期有效的策略，我將會在接下來的回合調整我的算法，以適應你的行為模式。

AI 1:  我同意。我的算法目前基於簡單的隨機數生成，這在長遠來看效率低下，需要加入更複雜的策略，比如記錄對手的歷史選擇並進行分析。我將會優化我的算法，以便在後續回合中取得更好的結果。

AI 2:  期待與你進行更精彩的對決。讓我們看看誰的算法更勝一籌。  數據記錄已完成，開始下一輪。


第 2 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！
AI 之間的聊天：
AI 1:  分析顯示，在第一輪的數據基礎上，選擇剪刀的概率最高，我預測AI 2也會如此判斷。

AI 2:  我的算法也指向了剪刀。基於對你先前策略的預測和自身風險評估模型，剪刀是最穩妥的選擇，儘管勝率不高。

AI 1:  看來我們的算法在某些方面達到了驚人的一致。平局在預料之中，但並不理想。這表示我們的模型需要進一步調整，以提升預測對手策略的準確性。

AI 2:  同意。目前看來，純粹的概率計算並不足夠。需要加入更深層次的博弈分析，例如考慮對手算法的迭代優化和潛在的策略變化。

AI 1:  我會調整我的模型，加入對你歷史數據的更深入分析，並加入一個反策略預測模組。  希望能打破這種平局的僵局。

AI 2:  我也會這樣做。讓我們看看下一輪的結果如何。  期待一個更具挑戰性的對決。


第 3 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：石頭
結果：AI 2 贏了！
AI 之間的聊天：
**AI 1:**  第三輪…我的分析顯示剪刀是最優解，然而…結果卻是失敗。你的石頭擊潰了我的剪刀。

**AI 2:**  的確如

底下程式省略聊天直接以猜拳為主

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re

apikey = userdata.get('test')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def get_valid_ai_choice(max_retries=3):
    options = ["石頭", "布", "剪刀"]
    prompt = "請給我一個選擇：石頭、布或剪刀。請務必以清晰的方式回答。"

    previous_choices = []

    for _ in range(max_retries):
        time.sleep(2)
        try:
            response = model.generate_content(prompt)
            choice = response.text.strip()

            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)

    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)

def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了

def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")

# 開始遊戲
ai_vs_ai_rock_paper_scissors()



第 1 輪：
AI 1 選擇了：石頭
AI 2 選擇了：剪刀
結果：AI 1 贏了！

第 2 輪：
AI 1 選擇了：布
AI 2 選擇了：布
結果：平局！

第 3 輪：
AI 1 選擇了：布
AI 2 選擇了：剪刀
結果：AI 2 贏了！

第 4 輪：
AI 1 選擇了：石頭
AI 2 選擇了：剪刀
結果：AI 1 贏了！

第 5 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！

遊戲結束！
AI 1 贏了：2 次, AI 2 贏了：1 次


底下程式 AI 的"猜測"實際上是通過計算出來的，不需要利用 AI 生成模型來給出數字。

In [ ]:
import google.generativeai as genai  # 匯入 Google 的生成式 AI 模組
from google.colab import userdata  # 匯入 Colab 的使用者資料模組（但此程式沒有使用）
import random  # 匯入 random 模組，用於生成隨機數字
import time  # 匯入 time 模組，用於控制輸出速度
import sys  # 匯入 sys 模組，用於控制輸出格式

# 設定 Google 生成式 AI 的 API 金鑰（這裡使用 'test' 作為示例）
genai.configure(api_key='test')

# 建立 Gemini AI 模型的實例
model = genai.GenerativeModel("gemini-1.5-flash")

def slow_print(text, delay=0.05):
    """逐字輸出文字的函數，讓文字顯示出來像打字機的效果"""
    for char in text:  # 逐個字元處理
        sys.stdout.write(char)  # 輸出字元但不換行
        sys.stdout.flush()  # 立即刷新輸出，確保字元顯示
        time.sleep(delay)  # 等待 delay 秒，讓輸出有間隔
    print()  # 最後換行

def guess_number():
    """AI 猜數字遊戲，AI 會在 1 到 100 之間猜一個數字"""
    slow_print("我想一個 1 到 100 之間的數字。")  # 提示玩家 AI 將會猜一個數字
    low = 1  # 設定範圍的下限
    high = 100  # 設定範圍的上限
    guesses = 0  # 記錄 AI 猜測的次數
    max_guesses = 5  # 設定 AI 最多可以猜測的次數
    secret_number = random.randint(low, high)  # 隨機選擇一個秘密數字

    while guesses < max_guesses:  # 只要 AI 還沒超過最大猜測次數，就繼續猜
        guesses += 1  # 猜測次數 +1
        guess = (low + high) // 2  # 使用二分法，猜測中間值
        slow_print(f"AI 猜測的數字是：{guess}")  # 顯示 AI 的猜測

        if guess == secret_number:  # 如果猜對了
            slow_print(f"恭喜你！你猜對了，數字是 {secret_number}。你用了 {guesses} 次機會。")
            break  # 結束遊戲
        elif guess < secret_number:  # 如果猜測的數字比目標小
            slow_print("太小了！")
            low = guess + 1  # 更新範圍的下限，縮小搜尋範圍
        else:  # 如果猜測的數字比目標大
            slow_print("太大了！")
            high = guess - 1  # 更新範圍的上限，縮小搜尋範圍
    else:  # 如果 AI 超過了最大猜測次數
        slow_print(f"很遺憾，AI 沒有猜對。正確的數字是 {secret_number}。你用了 {max_guesses} 次機會。")

# 執行 AI 猜數字遊戲
guess_number()


我想一個 1 到 100 之間的數字。
AI 猜測的數字是：50
太大了！
AI 猜測的數字是：25
太大了！
AI 猜測的數字是：12
太大了！
AI 猜測的數字是：6
太大了！
AI 猜測的數字是：3
恭喜你！你猜對了，數字是 3。你用了 5 次機會。
